# gifrer促销响应预测
## 1. 基于复购周期维度
    1.1 购买2次以上老客的回购周期计算
    1.2 只买过1次的新客的回购周期预测(根据客单价和产品及折扣率)
## 2. 基于产品关联维度
    2.1 粉色鼻滴和其他产品的关联度
    2.2 鼻喷和其他产品的关联度
## 3. 基于客户地域
    3.1 不同地域的回购率比较
    3.2 不同地域的产品偏好
    
## 4. 基于提升树算法的响应预测

In [79]:
'''备注，产品数字代号
1.粉色鼻滴
2.鼻喷
3.10支鼻滴
4.感冒鼻滴
5.鼻炎鼻滴
6.洗沐二合一
7.爽身粉
8.护臀
0.有价券
9.福袋
'''

'\n1.鼻滴\n2.鼻喷\n3.10支鼻滴\n4.感冒\n5.鼻炎\n6.洗沐二合一\n7.爽身粉\n8.护臀\n0.有价券\n9.福袋\n'

## 数据导入及清洗

In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False#是否可以显示负号

In [81]:
order_df=pd.read_csv('/Users/xucoolboy/practise/gifrer/gifrer海外旗舰店_utf8.csv')
cus_df=pd.read_csv('/Users/xucoolboy/practise/gifrer/gifrer_cus_utf8.csv')

In [82]:
order_df.head()

,订单号,宝贝标题,宝贝链接,子订单金额,成交单价,宝贝数量,子订单金额（折算:邮费、优惠等）,买家旺旺,付款时间,创建时间,SKU名称,来源
0,2238336379574953,Gifrer肌肤蕾法国生理盐水滴鼻剂婴儿宝宝通鼻塞液洗鼻通气海盐水,http://item.taobao.com/item.htm?id=538210886460,69.8,69.8,1,69.8,钱喵喵的口袋,2016/10/7 6:40,2016/10/7 6:40,NaN,无线
1,2489373647833634,10元优惠券，拍下联系客服,http://item.taobao.com/item.htm?id=539698601172,1.0,1.0,1,1.0,scorpin21,2016/10/8 9:33,2016/10/8 9:33,NaN,无线
2,2490241036301907,10元优惠券，拍下联系客服,http://item.taobao.com/item.htm?id=539698601172,1.0,1.0,1,1.0,细末灰尘,2016/10/8 11:51,2016/10/8 11:51,NaN,无线
3,2395374681200879,10元优惠券，拍下联系客服,http://item.taobao.com/item.htm?id=539698601172,1.0,1.0,1,1.0,leezhh华华,2016/10/8 11:53,2016/10/8 11:52,NaN,无线
4,2491386050130728,10元优惠券，拍下联系客服,http://item.taobao.com/item.htm?id=539698601172,1.0,1.0,1,1.0,恋上一滴泪922,2016/10/8 12:12,2016/10/8 12:12,NaN,无线


In [83]:
cus_df.head()

,买家昵称,买家姓名,印象分,性别,生日,手机号,邮箱,QQ,微信,会员状态,...,累计消费金额,成功交易笔数,平均客单价,购买宝贝件数,交易关闭笔数,交易关闭金额,未购买时间,省份,城市,收货地址
0,warjasonoz,洪建成,0,未知,NaN,13713214322,317379964@qq.com,NaN,NaN,正常,...,67.21,1,67.21,1,0,0.00,12.0天,广东省,东莞市,中堂镇潢涌村东莞市建桦造纸有限公司
1,我是定伟,方芸,0,未知,NaN,13275181683,NaN,NaN,NaN,正常,...,0.00,0,0.00,0,1,175.70,12.0天,江苏省,扬州市,氾水镇成庄村五里组19号
2,qq276148623,温春蕾,0,未知,NaN,18868258287,ufo888ufo@yahoo.com.cn,NaN,NaN,正常,...,0.00,0,0.00,0,1,28.00,12.0天,浙江省,温州市,松台街道勤奋路河通锦园5栋304
3,阿蓁小糯米团子,李双,0,未知,NaN,18682699772,NaN,NaN,NaN,正常,...,0.00,0,0.00,0,3,286.34,12.0天,四川省,成都市,华阳镇街道南湖国际三期九栋二单元（易泊：五期晟湖26栋)
4,susie_hkvc,yanglin,0,未知,NaN,13988548202,491306235@qq.com,NaN,NaN,正常,...,0.00,0,0.00,0,1,98.97,12.0天,云南省,大理白族自治州,下关镇文化路 惠丰新城诏园1-707


In [84]:
#时间格式转换
order_df['付款时间']=pd.to_datetime(order_df.付款时间)
order_df['创建时间']=pd.to_datetime(order_df.创建时间)
cus_df['淘宝注册时间']=pd.to_datetime(cus_df.淘宝注册时间)
cus_df['最近交易时间']=pd.to_datetime(cus_df.最近交易时间)

In [85]:
order_df['订单号']=order_df['订单号'].astype('str')
order_df.insert(1,'订单标识',order_df['订单号'].str[0:8]+order_df['订单号'].str[-4:])

In [86]:
order_df['付款年月']=order_df.付款时间.values.astype('datetime64[M]')
order_df['宝贝id']=order_df.宝贝链接.str.extract('=(\d+)')

/Users/xucoolboy/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  from ipykernel import kernelapp as app


In [87]:
#由于很多是重复链接，或者组合链接，所以导出所有宝贝id，浏览器打开链接查看商品种类，在csv中补全
pd.DataFrame(order_df.宝贝链接.str.extract('=(\d+)').value_counts()).to_csv('baobeiid')

/Users/xucoolboy/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  from ipykernel import kernelapp as app


In [88]:
#补全类型信息后导入csv文件
baobeitype_df=pd.read_csv('baobeiidnew')
baobeitype_df.columns=['宝贝id','类型']
baobeitype_df.head()
baobeitype_df['类型']=baobeitype_df['类型'].astype('str')

In [89]:
order_df['宝贝id']=order_df['宝贝id'].astype('int')

In [90]:
order_df['宝贝id'].head()

0    538210886460
1    539698601172
2    539698601172
3    539698601172
4    539698601172
Name: 宝贝id, dtype: int64

In [91]:
baobeitype_df['宝贝id'].dtype

dtype('int64')

In [92]:
order_df['宝贝id'].dtype

dtype('int64')

In [93]:
order_df=order_df.merge(baobeitype_df,on='宝贝id')

In [94]:
order_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136085 entries, 0 to 136084
Data columns (total 16 columns):
订单号                 136085 non-null object
订单标识                136085 non-null object
宝贝标题                135840 non-null object
宝贝链接                136085 non-null object
子订单金额               136085 non-null float64
成交单价                136085 non-null float64
宝贝数量                136085 non-null int64
子订单金额（折算:邮费、优惠等）    136085 non-null float64
买家旺旺                136084 non-null object
付款时间                136085 non-null datetime64[ns]
创建时间                136085 non-null datetime64[ns]
SKU名称               10412 non-null object
来源                  136085 non-null object
付款年月                136085 non-null datetime64[ns]
宝贝id                136085 non-null int64
类型                  136085 non-null object
dtypes: datetime64[ns](3), float64(3), int64(2), object(8)
memory usage: 17.7+ MB


In [95]:
order_df=order_df.groupby('订单标识').agg({'买家旺旺':'max','付款时间':'max','付款年月':'max','子订单金额':'sum',
                             '宝贝数量':'sum','类型':'sum'}).reset_index()

In [96]:
order_df.head()

,订单标识,买家旺旺,付款时间,付款年月,子订单金额,宝贝数量,类型
0,100000237543,瑞哥嚣张瑞瑞,2017-12-20 16:29:00,2017-12-01,128.00,2,12
1,100001060610,花间小妖f,2017-12-01 12:02:00,2017-12-01,144.35,1,14
2,100001388032,伊莜语,2017-12-20 16:25:00,2017-12-01,61.02,1,1
3,100001413453,多思思1,2017-12-20 16:29:00,2017-12-01,128.00,2,12
4,100002980843,riverredwave,2017-12-20 16:36:00,2017-12-01,61.02,1,1


## 基于复购周期维度

In [169]:
#统计各买家生命周期
user_life_ss=(order_df.groupby('买家旺旺').付款时间.apply(lambda x:x.max()-x.min()))/np.timedelta64(1,'D')
#统计各买家支付金额
user_order_amount_ss=(order_df.groupby('买家旺旺').子订单金额.sum().reset_index())
#统计各买家订单数
order_df_unigroup=order_df.drop_duplicates('订单标识').groupby('买家旺旺')
user_order_num_ss=order_df_unigroup.订单标识.count()
#统计各买家购买件数
user_product_num_ss=order_df.groupby('买家旺旺').宝贝数量.sum()
#统计各买家最近购买时间间隔
user_recent_ss=order_df.groupby('买家旺旺').付款时间.max()
user_recent_ss=(user_recent_ss.max()-user_recent_ss)/np.timedelta64(1,'D')

In [170]:
#合并买家生命周期，支付总金额、订单数、购买宝贝数
cus_info_total=pd.concat([user_life_ss,user_recent_ss,user_order_amount_ss.set_index('买家旺旺'),
                          user_order_num_ss,user_product_num_ss],axis=1)
cus_info_total.columns=['生命周期','最近购买间隔','支付总金额','总购买次数','总购买件数']
cus_info_total.head()

,生命周期,最近购买间隔,支付总金额,总购买次数,总购买件数
买家旺旺,,,,,
0000nancy6,15.313889,139.965278,180.72,2,3
0000飞0000,0.000000,333.436806,77.21,1,1
000618hua,0.000000,535.584028,38.05,1,1
000dandan87,0.000000,186.543750,77.21,1,1
000littlejany,0.000000,244.031250,60.00,1,2


In [171]:
#计算各买家平均指标
cus_info_total['生命周期区间']=pd.cut(cus_info_total.生命周期,bins=[-0.5,1.5,30,90,180,360,720])
cus_info_total['平均购买间隔']=cus_info_total['生命周期']/(cus_info_total['总购买次数']-1)
cus_info_total['平均购买间隔区间']=pd.cut(cus_info_total.平均购买间隔,bins=[-0.5,1.5,30,90,180,360,720])
cus_info_total['订单单价']=cus_info_total['支付总金额']/cus_info_total['总购买次数']

In [ ]:
### 有过回购客户筛选

In [172]:
cus_info_total[cus_info_total.生命周期>1.5].head()

,生命周期,最近购买间隔,支付总金额,总购买次数,总购买件数,生命周期区间,平均购买间隔,平均购买间隔区间,订单单价
买家旺旺,,,,,,,,,
0000nancy6,15.313889,139.965278,180.72,2,3,"(1.5, 30]",15.313889,"(1.5, 30]",90.360000
0105nina,110.745833,28.388194,116.21,2,2,"(90, 180]",110.745833,"(90, 180]",58.105000
0121sa,2.769444,345.986806,423.19,3,8,"(1.5, 30]",1.384722,"(-0.5, 1.5]",141.063333
025tf,30.347222,314.168750,185.21,2,3,"(30, 90]",30.347222,"(30, 90]",92.605000
0411ysj,333.319444,12.675000,278.84,2,4,"(180, 360]",333.319444,"(180, 360]",139.420000


In [ ]:
唤醒优先级 最近购买间隔在平均购买间隔前后15天>购买间隔大于平均购买间隔15-30天>购买间隔大于平均购买间隔30-60天>...
在此基础上考核买家购买力:平均单次购买折扣率和本次活动折扣率相比较，需要的话可以定向发券

In [179]:
cus_info_total[(cus_info_total.最近购买间隔<cus_info_total.平均购买间隔+15)&\
               (cus_info_total.平均购买间隔-15<cus_info_total.最近购买间隔)&\
              (cus_info_total.平均购买间隔>=15)]

,生命周期,最近购买间隔,支付总金额,总购买次数,总购买件数,生命周期区间,平均购买间隔,平均购买间隔区间,订单单价
买家旺旺,,,,,,,,,
123小苛,225.550694,219.572222,190.44,2,3,"(180, 360]",225.550694,"(180, 360]",95.220000
136内衣,38.768750,43.255556,167.40,2,2,"(30, 90]",38.768750,"(30, 90]",83.700000
13883270113liu,69.256944,82.301389,196.21,2,2,"(30, 90]",69.256944,"(30, 90]",98.105000
1825yuxin,146.102083,87.572917,230.06,3,5,"(90, 180]",73.051042,"(30, 90]",76.686667
26766650爽,41.075694,53.443750,105.26,2,2,"(30, 90]",41.075694,"(30, 90]",52.630000
2fei888,69.998611,66.329167,237.59,2,3,"(30, 90]",69.998611,"(30, 90]",118.795000
2qimin,185.415972,34.161806,368.42,6,9,"(180, 360]",37.083194,"(30, 90]",61.403333
539爱华晨宇,174.576389,170.899306,137.02,2,2,"(90, 180]",174.576389,"(90, 180]",68.510000
566zhushuwen,218.638889,120.579167,868.69,3,9,"(180, 360]",109.319444,"(90, 180]",289.563333


In [ ]:
### 只购买过一次客户筛选

In [173]:
#计算不同订单单价下的客户在不同购买间隔区间的分布
cus_info_total.groupby('平均购买间隔区间').apply(lambda x:x.订单单价).unstack().T.\
apply(lambda x:pd.cut(x,bins=[0,50,100,150,200,250,2000])).apply(lambda x:pd.value_counts(x)).T

,"(0, 50]","(100, 150]","(150, 200]","(200, 250]","(250, 2000]","(50, 100]"
平均购买间隔区间,,,,,,
"(-0.5, 1.5]",294,385,63,15,8,898
"(1.5, 30]",351,544,117,44,24,1228
"(30, 90]",257,691,203,63,39,1414
"(90, 180]",111,439,107,20,15,1032
"(180, 360]",82,224,44,9,7,603
"(360, 720]",29,13,3,1,0,58


In [198]:
#只购买了一次的客户(包括了15天)
cus_info_total[(cus_info_total.平均购买间隔<=15)|(cus_info_total.平均购买间隔.isnull())]

,生命周期,最近购买间隔,支付总金额,总购买次数,总购买件数,生命周期区间,平均购买间隔,平均购买间隔区间,订单单价
买家旺旺,,,,,,,,,
0000飞0000,0.0,333.436806,77.21,1,1,"(-0.5, 1.5]",NaN,NaN,77.21
000618hua,0.0,535.584028,38.05,1,1,"(-0.5, 1.5]",NaN,NaN,38.05
000dandan87,0.0,186.543750,77.21,1,1,"(-0.5, 1.5]",NaN,NaN,77.21
000littlejany,0.0,244.031250,60.00,1,2,"(-0.5, 1.5]",NaN,NaN,60.00
000maggie,0.0,135.987500,77.21,1,1,"(-0.5, 1.5]",NaN,NaN,77.21
000太婆,0.0,345.813194,154.42,1,2,"(-0.5, 1.5]",NaN,NaN,154.42
000波波,0.0,398.265972,66.02,1,1,"(-0.5, 1.5]",NaN,NaN,66.02
000王珏000,0.0,73.409028,79.00,1,1,"(-0.5, 1.5]",NaN,NaN,79.00
000瑶瑶,0.0,103.585417,178.00,1,2,"(-0.5, 1.5]",NaN,NaN,178.00


In [ ]:
唤醒优先级 根据所在客单价区间的购买间隔平均数或者中位数预测可能的购买间隔

In [ ]:
写两个for循环算出新客的客单价在老客订单单价区间中的中位数或者平均数

## 基于产品关联维度

In [97]:
#计算出最早购买时间表(不包括有价券)
cus_first_df=pd.DataFrame(order_df[order_df.类型!=0].groupby('买家旺旺').付款时间.min()).reset_index()

In [98]:
cus_first_df.head()

,买家旺旺,付款时间
0,0000nancy6,2017-11-26 17:14:00
1,0000飞0000,2017-06-01 13:27:00
2,000618hua,2016-11-11 09:55:00
3,000dandan87,2017-10-26 10:53:00
4,000littlejany,2017-08-29 23:11:00


In [99]:
#顾客最早购买订单详情
cus_firstinfo_df=order_df.merge(cus_first_df,on=['买家旺旺','付款时间'])

In [100]:
#计算买家回购的订单
cus_notfirst_df=order_df[(~(order_df.买家旺旺+order_df.付款时间.astype('str')).\
isin(cus_first_df.买家旺旺+cus_first_df.付款时间.astype('str')))&(order_df.类型!=0)]

In [101]:
cus_notfirst_df.head()

,订单标识,买家旺旺,付款时间,付款年月,子订单金额,宝贝数量,类型
26,100036272248,wanghuiyuyi,2017-12-20 20:54:00,2017-12-01,61.02,1,1
40,100052038952,子木一诺,2017-12-20 19:30:00,2017-12-01,61.02,1,1
59,100076420431,tb0066979_55,2017-12-20 20:51:00,2017-12-01,61.02,1,1
66,100081978150,止水南开,2017-12-20 21:10:00,2017-12-01,140.00,2,715
143,100167875859,yizhihaome,2017-12-21 11:22:00,2017-12-01,61.02,1,1


In [102]:
#计算购买过实物的总人数
cus_first_df.shape

(109902, 2)

In [103]:
#计算购买过实物的总人数(不去除支付时间相同的不同订单)
cus_firstinfo_df.买家旺旺.count()

110101

In [104]:
#计算第一次订单中包含粉色鼻滴的人数
cus_firstinfo_df[cus_firstinfo_df.类型.str.contains('1|3')].买家旺旺.count()

81522

In [105]:
#计算第一次订单中包含鼻喷的人数
cus_firstinfo_df[cus_firstinfo_df.类型.str.contains('2')].买家旺旺.count()

23310

In [106]:
#计算第一次订单中包含感冒盐水的人数
cus_firstinfo_df[cus_firstinfo_df.类型.str.contains('4')].买家旺旺.count()

4884

In [107]:
#计算第一次订单中包含鼻炎盐水的人数
cus_firstinfo_df[cus_firstinfo_df.类型.str.contains('5')].买家旺旺.count()

4945

In [110]:
#计算第一次订单中同时包含鼻喷和粉色鼻滴的人数
cus_firstinfo_df[(cus_firstinfo_df.类型.str.contains('2'))&\
                (cus_firstinfo_df.类型.str.contains('1|3'))].买家旺旺.count()

3888

In [150]:
#计算第一次订单中同时包含感冒盐水和粉色鼻滴的人数
cus_firstinfo_df[(cus_firstinfo_df.类型.str.contains('4'))&\
                (cus_firstinfo_df.类型.str.contains('1|3'))].买家旺旺.count()

2767

In [151]:
#计算第一次订单中同时包含鼻炎盐水和粉色鼻滴的人数
cus_firstinfo_df[(cus_firstinfo_df.类型.str.contains('5'))&\
                (cus_firstinfo_df.类型.str.contains('1|3'))].买家旺旺.count()

3172

In [111]:
#计算回购总人数
cus_notfirst_df.groupby('买家旺旺').类型.count().count()

9282

### 第一次购买各主推产品的置信度和提升度计算

In [146]:
#鼻喷对粉色鼻滴的置信度
3888/23310

0.1667953667953668

In [147]:
#粉色鼻滴对鼻喷的置信度
3888/81522

0.04769264738352837

In [148]:
#鼻喷对粉色鼻滴的提升度
(3888/23310)/(81522/110101)

0.22526847574319422

In [152]:
#感冒盐水对粉色鼻滴的置信度
2767/4884

0.5665438165438166

In [153]:
#鼻炎盐水对粉色鼻滴的置信度
3172/4945

0.6414560161779576

In [154]:
#感冒盐水对粉色鼻滴的提升度
(2767/4884)/(81522/110101)

0.765155917976629

In [155]:
#鼻炎盐水对粉色鼻滴的提升度
(3172/4945)/(81522/110101)

0.866329933480647

### 第一次购买哪些产品的人，下一次回购会购买鼻喷的可能性最大

In [140]:
#计算第一次订单中包含鼻喷，有回购的人数
cus_firstinfo_df[
                (cus_firstinfo_df.类型.str.contains('2'))]\
                .merge(cus_notfirst_df,on='买家旺旺')\
                .groupby('买家旺旺').类型_x.count().count()

2691

In [141]:
#计算第一次订单中包含鼻喷，回购订单中包含鼻喷的人数
cus_firstinfo_df[
                (cus_firstinfo_df.类型.str.contains('2'))]\
                .merge(cus_notfirst_df[cus_notfirst_df.类型.str.contains('2')],on='买家旺旺')\
                .groupby('买家旺旺').类型_x.count().count()

1925

In [142]:
#第一次购买鼻喷，下次购买鼻喷可能性
1925/2691

0.7153474544778893

In [112]:
#计算第一次订单中包含粉色鼻滴但不包含鼻喷，有回购的人数
cus_firstinfo_df[(cus_firstinfo_df.类型.str.contains('1|3'))&\
                (~cus_firstinfo_df.类型.str.contains('2'))]\
                .merge(cus_notfirst_df,on='买家旺旺')\
                .groupby('买家旺旺').类型_x.count().count()

4966

In [113]:
#计算第一次订单中包含粉色鼻滴但不包含鼻喷，回购订单中包含鼻喷的人数
cus_firstinfo_df[(cus_firstinfo_df.类型.str.contains('1|3'))&\
                (~cus_firstinfo_df.类型.str.contains('2'))]\
                .merge(cus_notfirst_df[cus_notfirst_df.类型.str.contains('2')],on='买家旺旺')\
                .groupby('买家旺旺').类型_x.count().count()

1121

In [121]:
#第一次购买粉色鼻滴但不包含鼻喷，下次购买鼻喷可能性
1121/4966

0.2257349979863069

In [115]:
#计算第一次订单中包含感冒盐水但不包含鼻喷，有回购的人数
cus_firstinfo_df[(cus_firstinfo_df.类型.str.contains('4'))&\
                (~cus_firstinfo_df.类型.str.contains('2'))]\
                .merge(cus_notfirst_df,on='买家旺旺')\
                .groupby('买家旺旺').类型_x.count().count()

450

In [116]:
#计算第一次订单中包含感冒盐水但不包含鼻喷，回购订单中包含鼻喷的人数
cus_firstinfo_df[(cus_firstinfo_df.类型.str.contains('4'))&\
                (~cus_firstinfo_df.类型.str.contains('2'))]\
                .merge(cus_notfirst_df[cus_notfirst_df.类型.str.contains('2')],on='买家旺旺')\
                .groupby('买家旺旺').类型_x.count().count()

140

In [122]:
#第一次购买感冒盐水但不包含鼻喷，下次购买鼻喷可能性
140/450

0.3111111111111111

In [158]:
#计算第一次订单中包含鼻炎盐水但不包含鼻喷，有回购的人数
cus_firstinfo_df[(cus_firstinfo_df.类型.str.contains('5'))&\
                (~cus_firstinfo_df.类型.str.contains('2'))]\
                .merge(cus_notfirst_df,on='买家旺旺')\
                .groupby('买家旺旺').类型_x.count().count()

440

In [159]:
#计算第一次订单中包含感冒盐水但不包含鼻喷，回购订单中包含鼻喷的人数
cus_firstinfo_df[(cus_firstinfo_df.类型.str.contains('5'))&\
                (~cus_firstinfo_df.类型.str.contains('2'))]\
                .merge(cus_notfirst_df[cus_notfirst_df.类型.str.contains('2')],on='买家旺旺')\
                .groupby('买家旺旺').类型_x.count().count()

119

In [123]:
#第一次购买鼻炎盐水但不包含鼻喷，下次购买鼻喷可能性
119/440

0.27045454545454545

鼻喷单品团会员短信发送优先级:购买过鼻喷>购买过感冒盐水>购买过鼻炎盐水>购买过粉色鼻滴

### 第一次购买哪些产品的人，下一次回购会购买粉色鼻滴的可能性最大

In [143]:
#计算第一次订单中包含粉色鼻滴，有回购的人数
cus_firstinfo_df[
                (cus_firstinfo_df.类型.str.contains('1|3'))]\
                .merge(cus_notfirst_df,on='买家旺旺')\
                .groupby('买家旺旺').类型_x.count().count()

5400

In [144]:
#计算第一次订单中包含粉色鼻滴，回购粉色鼻滴的人数
cus_firstinfo_df[
                (cus_firstinfo_df.类型.str.contains('1|3'))]\
                .merge(cus_notfirst_df[cus_notfirst_df.类型.str.contains('1|3')],on='买家旺旺')\
                .groupby('买家旺旺').类型_x.count().count()

4084

In [145]:
#第一次购买粉色鼻滴，下次还购买粉色鼻滴的可能性
4084/5400

0.7562962962962962

In [124]:
#计算第一次订单中包含鼻喷但不包含粉色鼻滴，有回购的人数
cus_firstinfo_df[(cus_firstinfo_df.类型.str.contains('2'))&\
                (~cus_firstinfo_df.类型.str.contains('1|3'))]\
                .merge(cus_notfirst_df,on='买家旺旺')\
                .groupby('买家旺旺').类型_x.count().count()

2257

In [125]:
#计算第一次订单中包含鼻喷但不包含粉色鼻滴，回购订单中包含粉色鼻滴的人数
cus_firstinfo_df[(cus_firstinfo_df.类型.str.contains('2'))&\
                (~cus_firstinfo_df.类型.str.contains('1|3'))]\
                .merge(cus_notfirst_df[cus_notfirst_df.类型.str.contains('1|3')],on='买家旺旺')\
                .groupby('买家旺旺').类型_x.count().count()

674

In [126]:
#第一次购买鼻喷但不包含粉色鼻滴，下次购买粉色鼻滴的可能性
674/2257

0.29862649534780683

In [127]:
#计算第一次订单中包含感冒但不包含粉色鼻滴，有回购的人数
cus_firstinfo_df[(cus_firstinfo_df.类型.str.contains('4'))&\
                (~cus_firstinfo_df.类型.str.contains('1|3'))]\
                .merge(cus_notfirst_df,on='买家旺旺')\
                .groupby('买家旺旺').类型_x.count().count()

292

In [128]:
#计算第一次订单中包含感冒但不包含粉色鼻滴，回购订单中包含粉色鼻滴的人数
cus_firstinfo_df[(cus_firstinfo_df.类型.str.contains('4'))&\
                (~cus_firstinfo_df.类型.str.contains('1|3'))]\
                .merge(cus_notfirst_df[cus_notfirst_df.类型.str.contains('1|3')],on='买家旺旺')\
                .groupby('买家旺旺').类型_x.count().count()

149

In [129]:
#第一次购买感冒盐水但不包含粉色鼻滴，下次购买粉色鼻滴的可能性
149/292

0.5102739726027398

In [130]:
#计算第一次订单中包含鼻炎但不包含粉色鼻滴，有回购的人数
cus_firstinfo_df[(cus_firstinfo_df.类型.str.contains('5'))&\
                (~cus_firstinfo_df.类型.str.contains('1|3'))]\
                .merge(cus_notfirst_df,on='买家旺旺')\
                .groupby('买家旺旺').类型_x.count().count()

329

In [131]:
#计算第一次订单中包含鼻炎但不包含粉色鼻滴，回购订单中包含粉色鼻滴的人数
cus_firstinfo_df[(cus_firstinfo_df.类型.str.contains('5'))&\
                (~cus_firstinfo_df.类型.str.contains('1|3'))]\
                .merge(cus_notfirst_df[cus_notfirst_df.类型.str.contains('1|3')],on='买家旺旺')\
                .groupby('买家旺旺').类型_x.count().count()

133

In [132]:
#第一次购买鼻炎盐水但不包含粉色鼻滴，下次购买粉色鼻滴的可能性
133/329

0.40425531914893614

粉色鼻滴单品团会员短信发送优先级:购买过粉色鼻滴>购买过感冒盐水>购买过鼻炎盐水>购买过鼻喷

之所以鼻炎盐水对粉色鼻滴的置信度高于感冒盐水对粉色鼻滴的置信度，但从下一次回购粉色盐水可能性来看是感冒盐水大于鼻炎盐水，   
是因为鼻炎盐水和粉色鼻滴差异较大，所以第一次购买要搭配买家肯定优先搭配差异大的，而第一次选择了感冒盐水的人使用后觉得差异不大  
可能会将粉色盐水作为替代品，鼻炎盐水和粉色鼻滴差异大，很难作为替代品。

In [ ]:
如果是整店活动，只需考虑客户购买周期

## 基于客户地域

In [199]:
order_df=order_df.merge(cus_df[['买家昵称','省份']],left_on='买家旺旺',right_on='买家昵称').drop('买家昵称',axis=1)

In [200]:
#计算各省份支付金额
sale_diyu_df=pd.concat([order_df[order_df.付款年月=='2017-12-01'].groupby('省份').子订单金额.sum(),
order_df[order_df.付款年月=='2018-01-01'].groupby('省份').子订单金额.sum()],axis=1)
sale_diyu_df.columns=['12月','1月']

In [201]:
sale_diyu_df.sort_values('12月',ascending=False)

,12月,1月
省份,,
广东省,160299.79,149220.13
浙江省,100863.72,106689.72
上海,85249.20,78590.92
江苏省,82100.05,81063.33
北京,63690.83,55449.72
山东省,56144.94,60743.02
福建省,48913.64,53502.52
湖北省,47913.44,51533.28
四川省,39903.93,41298.22


In [202]:
#计算各省份的购买订单数
ordernum_diyu_df=pd.concat([order_df[order_df.付款年月=='2017-12-01'].groupby('省份').子订单金额.count(),
order_df[order_df.付款年月=='2018-01-01'].groupby('省份').子订单金额.count()],axis=1)
ordernum_diyu_df.columns=['12月','1月']

In [203]:
#计算各省份的订单均价
bidanjia_diyu_df=sale_diyu_df/ordernum_diyu_df
bidanjia_diyu_df['降幅']=(bidanjia_diyu_df['12月']-bidanjia_diyu_df['1月'])/bidanjia_diyu_df['12月']
bidanjia_diyu_df.sort_values('降幅',ascending=False)

,12月,1月,降幅
省份,,,
西藏自治区,107.164000,71.615000,0.331725
天津,99.591245,79.445197,0.202287
河北省,85.067507,68.387330,0.196082
贵州省,85.025769,69.417681,0.183569
山东省,81.134306,66.971356,0.174562
浙江省,85.333096,70.515347,0.173646
青海省,84.103611,69.514706,0.173463
甘肃省,83.463540,69.528208,0.166963
辽宁省,78.482432,65.571496,0.164507


In [204]:
pivoted_user_buy=order_df.pivot_table(index='买家旺旺',
                              columns='付款年月',
                              values='订单标识',
                              aggfunc='count')
pivoted_user_buy=pivoted_user_buy.applymap(lambda x:1 if x>1 else 0 if x==1 else np.NaN)

In [205]:
#计算是否有回购函数
def huigou_after(x):
    L=[]
    for i in range((len(x)-1)):
        if x[i]>=0:
            for j in range(i+1,len(x)):
                if x[j]>=0:
                    L.append(1)
                    break
                else:
                    if j==len(x)-1:
                        L.append(0)
        else:
            L.append(np.NaN)
    if x[len(x)-1]>=0:
        L.append(0)
    else:
        L.append(np.NaN)
    return(L)

In [206]:
#得出各月各个买家是否回购
huigou_after_df=pivoted_user_buy.apply(lambda x:huigou_after(x),axis=1)
huigou_after_df=huigou_after_df.merge(cus_df[['买家昵称','省份']],left_index=True,right_on='买家昵称')

In [207]:
#各月各省留存人数
huigou_diyu_df=huigou_after_df.groupby('省份').sum()

In [208]:
#各月各省购买人数
goumai_df=pivoted_user_buy.merge(cus_df[['买家昵称','省份']],left_index=True,right_on='买家昵称')\
.groupby('省份').count()

In [209]:
#各月各省留存率指标
(huigou_diyu_df.iloc[:,0:19])/(goumai_df.iloc[:,0:19])

,2016-10-01 00:00:00,2016-11-01 00:00:00,2016-12-01 00:00:00,2017-01-01 00:00:00,2017-02-01 00:00:00,2017-03-01 00:00:00,2017-04-01 00:00:00,2017-05-01 00:00:00,2017-06-01 00:00:00,2017-07-01 00:00:00,2017-08-01 00:00:00,2017-09-01 00:00:00,2017-10-01 00:00:00,2017-11-01 00:00:00,2017-12-01 00:00:00,2018-01-01 00:00:00,2018-02-01 00:00:00,2018-03-01 00:00:00,2018-04-01 00:00:00
省份,,,,,,,,,,,,,,,,,,,
上海,NaN,1.0,1.000000,1.0,1.000000,1.000000,1.000000,0.965517,1.000000,1.000000,1.000000,0.975610,1.000000,0.132955,0.066059,0.046537,0.047619,0.016456,0.0
云南省,1.0,NaN,1.000000,1.0,1.000000,1.000000,0.833333,1.000000,1.000000,1.000000,1.000000,1.000000,0.916667,0.111111,0.046610,0.044983,0.042553,0.012245,0.0
内蒙古自治区,NaN,NaN,NaN,NaN,1.000000,1.000000,1.000000,0.875000,1.000000,1.000000,1.000000,0.600000,1.000000,0.095238,0.041096,0.013245,0.039474,0.005952,0.0
北京,NaN,1.0,0.666667,1.0,1.000000,1.000000,0.950000,1.000000,1.000000,1.000000,1.000000,0.935484,1.000000,0.092409,0.049844,0.040724,0.056250,0.015198,0.0
吉林省,NaN,1.0,NaN,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,NaN,1.000000,1.000000,1.000000,0.054688,0.043478,0.038043,0.052083,0.008000,0.0
四川省,NaN,1.0,1.000000,NaN,1.000000,1.000000,1.000000,0.952381,1.000000,0.900000,1.000000,1.000000,1.000000,0.077364,0.082192,0.046729,0.034921,0.009281,0.0
天津,NaN,NaN,NaN,1.0,1.000000,1.000000,1.000000,1.000000,0.909091,1.000000,1.000000,1.000000,1.000000,0.103448,0.046693,0.036585,0.025510,0.004464,0.0
宁夏回族自治区,NaN,NaN,NaN,1.0,1.000000,NaN,1.000000,1.000000,1.000000,NaN,NaN,1.000000,1.000000,0.117647,0.000000,0.051282,0.040000,0.000000,0.0
安徽省,0.0,1.0,1.000000,NaN,1.000000,0.875000,1.000000,1.000000,1.000000,0.888889,0.800000,1.000000,0.900000,0.108974,0.061111,0.045064,0.020408,0.011364,0.0


In [210]:
#统计客户购买月份数
newhuigou_df=order_df.groupby(['买家旺旺','付款年月']).宝贝数量.count().reset_index().\
groupby('买家旺旺').agg({'付款年月':'min','宝贝数量':'count'})
newhuigou_df.columns=['最早购买年月','总购买月份数']
newhuigou_df=newhuigou_df.merge(cus_df[['买家昵称','省份']],left_index=True,right_on='买家昵称')
newhuigou_df['是否回购']=newhuigou_df.总购买月份数.apply(lambda x:1 if x>1 else 0)

In [212]:
#统计各月分地域新客数
newcus_diyu_per_M=newhuigou_df.pivot_table(index='省份',columns='最早购买年月',values='是否回购',aggfunc='count')

In [213]:
#统计各月分地域回购新客数
newcushuigou_diyu_per_M=newhuigou_df.pivot_table(index='省份',columns='最早购买年月',values='是否回购',aggfunc='sum')

In [214]:
#统计各月新客回购比
newcushuigou_diyu_per_M/newcus_diyu_per_M

最早购买年月,2016-10-01 00:00:00,2016-11-01 00:00:00,2016-12-01 00:00:00,2017-01-01 00:00:00,2017-02-01 00:00:00,2017-03-01 00:00:00,2017-04-01 00:00:00,2017-05-01 00:00:00,2017-06-01 00:00:00,2017-07-01 00:00:00,2017-08-01 00:00:00,2017-09-01 00:00:00,2017-10-01 00:00:00,2017-11-01 00:00:00,2017-12-01 00:00:00,2018-01-01 00:00:00,2018-02-01 00:00:00,2018-03-01 00:00:00,2018-04-01 00:00:00
省份,,,,,,,,,,,,,,,,,,,
上海,NaN,1.0,1.000000,1.0,1.000000,1.000000,1.000000,0.960000,1.000000,1.000000,1.000000,1.000000,1.000000,0.105651,0.050441,0.032569,0.045652,0.008559,0.0
云南省,1.0,NaN,1.000000,1.0,1.000000,1.000000,0.800000,1.000000,1.000000,1.000000,1.000000,1.000000,0.857143,0.096386,0.040359,0.033210,0.030534,0.009302,0.0
内蒙古自治区,NaN,NaN,NaN,NaN,1.000000,1.000000,1.000000,0.833333,1.000000,1.000000,1.000000,0.600000,1.000000,0.090909,0.022727,0.007246,0.040541,0.006410,0.0
北京,NaN,1.0,0.666667,1.0,1.000000,1.000000,0.944444,1.000000,1.000000,1.000000,1.000000,0.904762,1.000000,0.078995,0.041026,0.030695,0.037671,0.011966,0.0
吉林省,NaN,1.0,NaN,1.0,1.000000,1.000000,1.000000,1.000000,NaN,NaN,1.000000,1.000000,1.000000,0.056000,0.037037,0.029240,0.043011,0.009009,0.0
四川省,NaN,1.0,1.000000,NaN,1.000000,1.000000,1.000000,0.947368,1.000000,0.857143,1.000000,1.000000,1.000000,0.065672,0.068127,0.036072,0.026936,0.007772,0.0
天津,NaN,NaN,NaN,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.081967,0.035398,0.031111,0.021858,0.004926,0.0
宁夏回族自治区,NaN,NaN,NaN,1.0,1.000000,NaN,1.000000,1.000000,NaN,NaN,NaN,1.000000,1.000000,0.093750,0.000000,0.029412,0.041667,0.000000,0.0
安徽省,0.0,1.0,1.000000,NaN,1.000000,0.833333,1.000000,1.000000,1.000000,0.875000,0.666667,1.000000,0.888889,0.093960,0.048193,0.029545,0.017621,0.009494,0.0


In [215]:
#各省客户数
newcushuigou_diyu_per_M.T.sum().astype('int')

省份
上海          360
云南省          86
内蒙古自治区       58
北京          252
吉林省          39
四川省         154
天津          102
宁夏回族自治区      14
安徽省         118
山东省         210
山西省          60
广东省         577
广西壮族自治区      62
新疆维吾尔自治区     44
江苏省         300
江西省          74
河北省         127
河南省         148
浙江省         398
海南省          33
湖北省         189
湖南省         136
甘肃省          37
福建省         230
西藏自治区         1
贵州省          49
辽宁省         105
重庆          110
陕西省          96
青海省          12
黑龙江省         55
dtype: int64

In [216]:
#各省回购客户数
newcus_diyu_per_M.T.sum().astype('int')

省份
上海          4019
云南省         1142
内蒙古自治区       707
北京          2993
吉林省          698
四川省         2156
天津          1221
宁夏回族自治区      183
安徽省         1784
山东省         3324
山西省         1000
广东省         7712
广西壮族自治区     1176
新疆维吾尔自治区     526
江苏省         4811
江西省         1325
河北省         1774
河南省         2232
浙江省         5588
海南省          439
湖北省         2827
湖南省         1773
甘肃省          497
福建省         2882
西藏自治区         20
贵州省          872
辽宁省         1566
重庆          1220
陕西省         1383
青海省          142
黑龙江省         992
dtype: int64

In [217]:
#各省份客户留存率
((newcushuigou_diyu_per_M.T.sum().astype('int'))/(newcus_diyu_per_M.T.sum().\
                                                  astype('int'))).sort_values(ascending=False)

省份
重庆          0.090164
上海          0.089575
青海省         0.084507
北京          0.084196
新疆维吾尔自治区    0.083650
天津          0.083538
内蒙古自治区      0.082037
福建省         0.079806
湖南省         0.076706
宁夏回族自治区     0.076503
云南省         0.075306
海南省         0.075171
广东省         0.074818
甘肃省         0.074447
河北省         0.071590
四川省         0.071429
浙江省         0.071224
陕西省         0.069414
辽宁省         0.067050
湖北省         0.066855
河南省         0.066308
安徽省         0.066143
山东省         0.063177
江苏省         0.062357
山西省         0.060000
贵州省         0.056193
吉林省         0.055874
江西省         0.055849
黑龙江省        0.055444
广西壮族自治区     0.052721
西藏自治区       0.050000
dtype: float64

在客户量级相差无几的情况下，按照客户留存率优选买家留存率高的地区

### 不同地域的产品偏好

## 基于提升树算法的响应预测

In [1]:
# 获得最佳模型参数
def get_best_model(X, y):
    '''
    结合交叉检验得到不同参数下的分类模型结果
    :param X: 输入X（特征变量）
    :param y: 预测y（目标变量）
    :return: 特征选择模型对象
    '''
    transform = SelectPercentile(f_classif, percentile=50)  # 使用f_classif方法选择特征最明显的50%数量的特征
    model_adaboost = AdaBoostClassifier()  # 建立AdaBoostClassifier模型对象
    model_pipe = Pipeline(steps=[('ANOVA', transform), ('model_adaboost', model_adaboost)])  # 建立由特征选择和分类模型构成的“管道”对象
    cv = StratifiedKFold(5)  # 设置交叉检验次数
    n_estimators = [20, 50, 80, 100]  # 设置模型参数列表
    score_methods = ['accuracy', 'f1', 'precision', 'recall', 'roc_auc']  # 设置交叉检验指标
    mean_list = list()  # 建立空列表用于存放不同参数方法、交叉检验评估指标的均值列表
    std_list = list()  # 建立空列表用于存放不同参数方法、交叉检验评估指标的标准差列表
    for parameter in n_estimators:  # 循环读出每个参数值
        t1 = time.time()  # 记录训练开始的时间
        score_list = list()  # 建立空列表用于存放不同交叉检验下各个评估指标的详细数据
        print ('set parameters: %s' % parameter)  # 打印当前模型使用的参数
        for score_method in score_methods:  # 循环读出每个交叉检验指标
            model_pipe.set_params(model_adaboost__n_estimators=parameter)  # 通过“管道”设置分类模型参数
            score_tmp = cross_val_score(model_pipe, X, y, scoring=score_method, cv=cv)  # 使用交叉检验计算指定指标的得分
            score_list.append(score_tmp)  # 将交叉检验得分存储到列表
        score_matrix = pd.DataFrame(np.array(score_list), index=score_methods)  # 将交叉检验详细数据转换为矩阵
        score_mean = score_matrix.mean(axis=1).rename('mean')  # 计算每个评估指标的均值
        score_std = score_matrix.std(axis=1).rename('std')  # 计算每个评估指标的标准差
        score_pd = pd.concat([score_matrix, score_mean, score_std], axis=1)  # 将原始详细数据和均值、标准差合并
        mean_list.append(score_mean)  # 将每个参数得到的各指标均值追加到列表
        std_list.append(score_std)  # 将每个参数得到的各指标标准差追加到列表
        print (score_pd.round(2))  # 打印每个参数得到的交叉检验指标数据，只保留2位小数
        print ('-' * 60)
        t2 = time.time()  # 计算每个参数下算法用时
        tt = t2 - t1  # 计算时间间隔
        print ('time: %s' % str(tt))  # 打印时间间隔
    mean_matrix = np.array(mean_list).T  # 建立所有参数得到的交叉检验的均值矩阵
    std_matrix = np.array(std_list).T  # 建立所有参数得到的交叉检验的标准差矩阵
    mean_pd = pd.DataFrame(mean_matrix, index=score_methods, columns=n_estimators)  # 将均值矩阵转换为数据框
    std_pd = pd.DataFrame(std_matrix, index=score_methods, columns=n_estimators)  # 将均值标准差转换为数据框
    print ('Mean values for each parameter:')
    print (mean_pd)  # 打印输出均值矩阵
    print ('Std values for each parameter:')
    print (std_pd)  # 打印输出标准差矩阵
    print ('-' * 60)
    return transform

In [ ]:
# 数据应用
# 加载数据集
raw_data = pd.read_excel('order.xlsx', sheetname=0)  # 读出Excel的第一个sheet
X = raw_data.drop('response', axis=1)  # 分割X
y = raw_data['response']  # 分割y

# 数据审查和预处理
set_summary(raw_data)  # 基本状态查看
na_summary(raw_data)  # 缺失值审查
label_summary(raw_data)  # 类样本均衡均衡审查
X_t1 = na_replace(X)  # 替换缺失值
X_t2 = type_con(X_t1)  # 数据类型转换
X_new, enc = symbol_con(X_t2, enc_object=None, train=True)  # 将分类和顺序数据转换为标志

# 分类模型训练
transform = get_best_model(X_new, y)  # 获得最佳分类模型参数信息
transform.fit(X_new, y)  # 应用特征选择对象选择要参与建模的特征变量
X_final = transform.transform(X_new)  # 获得具有显著性特征的特征变量
final_model = AdaBoostClassifier(n_estimators=100)  # 从打印的参数均值和标准差信息中确定参数并建立分类模型对象
final_model.fit(X_final, y)  # 训练模型

# 新数据集做预测
new_data = pd.read_excel('order.xlsx', sheetname=1)  # 读取要预测的数据集
final_reponse = new_data['final_response']  # 获取最终的目标变量值
new_data = new_data.drop('final_response', axis=1)  # 获得预测的输入变量X
set_summary(new_data)  # 基本状态查看
na_summary(new_data)  # 缺失值审查
new_X_t1 = na_replace(new_data)  # 替换缺失值
new_X_t2 = type_con(new_X_t1)  # 数据类型转换
new_X_t3 = symbol_con(new_X_t2, enc_object=enc, train=False)  # 将分类和顺序数据转换为标志
new_X_final = transform.transform(new_X_t3)  # 对数据集做特征选择

# 输出预测值以及预测概率
predict_labels = pd.DataFrame(final_model.predict(new_X_final), columns=['labels'])  # 获得预测标签
predict_labels_pro = pd.DataFrame(final_model.predict_proba(new_X_final), columns=['pro1', 'pro2'])  # 获得预测概率
predict_pd = pd.concat((new_data, predict_labels, predict_labels_pro), axis=1)  # 将预测标签、预测数据和原始数据X合并
print ('Predict info')
print (predict_pd.head(2))  # 打印前2条结果
print ('-' * 60)

# 将预测结果写入Excel
writer = pd.ExcelWriter('order_predict_result.xlsx')  # 创建写入文件对象
predict_pd.to_excel(writer, 'Sheet1')  # 将数据写入sheet1
writer.save()  # 保存文件

# 后续--与实际效果的比较
print ('final accuracy: {0}'.format(accuracy_score(final_reponse, predict_labels)))